# Model Persistence with Keras

After training a model, it is desirable to have a way to persist the model for future use without having to retrain. $*$

#### Security & maintainability limitations

- Never load untrusted data as it could lead to malicious code being executed upon loading.
- While models saved using one version of a framewrok  might load in other versions, this is entirely unsupported and inadvisable. It should also be kept in mind that operations performed on such data could give different and unexpected results.

In order to rebuild a similar model with future versions of your framework, additional metadata should be saved along the model:

- The training data, e.g. a reference to an immutable snapshot
- The python source code used to generate the model
- The versions of scikit-learn and its dependencies
- The cross validation score obtained on the training data

This should make it possible to check that the cross-validation score is in the same range as before.

Since a model internal representation may be different on two different architectures, dumping a model on one architecture and loading it on another architecture is not supported.

$*$ From Scikit-Learn Documentation: https://scikit-learn.org/stable/modules/model_persistence.html


## ML workflow / Models pipeline

<img src="imgs/models_pipeline.png" />


## Distributed Systems with Machine Learning
<img src="imgs/ML_Systems.png" />





## Model Versioning:

- Git / Git LFS
- [DVC](https://dvc.org/)



### Ways to replicate an environment
Using Virtualenv or Conda we can recreate enviroments with specific versions of libraries and even Python interpreter

- `pip freeze` (requirements.txt)
- `conda list` (environment.yaml)
- Docker Container + requirements.txt/environment.yaml


## Versions:

In [1]:
import numpy
import keras
import tensorflow
import sklearn
import h5py
import yaml
import joblib

Using TensorFlow backend.
/Users/israel/anaconda3/envs/data_dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/israel/anaconda3/envs/data_dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/israel/anaconda3/envs/data_dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/isra

In [2]:
print('NumPy: %s'%numpy.__version__ )
print('TensorFlow: %s'%tensorflow.__version__)
print('Scikit-Learn:  %s'%sklearn.__version__)
print('Keras: %s'%keras.__version__)
print('HDF5: %s'%h5py.__version__)
print('PyYAML: %s'%yaml.__version__)
print('Joblib: %s'%joblib.__version__)

NumPy: 1.18.1
TensorFlow: 1.14.0
Scikit-Learn:  0.22.1
Keras: 2.3.1
HDF5: 2.10.0
PyYAML: 5.3.1
Joblib: 0.14.1


### Utilities

In [3]:
def create_model():
    """
    Create a Neural network and return the trained model with the used features to train
    
    Output: Features, Target and model
    """
    try:
        # Import tools
        from keras.models import Sequential
        from keras.layers import Dense
        from keras.models import model_from_yaml
        import numpy
        # fix random seed for reproducibility
        seed = 7
        numpy.random.seed(seed)
        # load pima indians dataset
        dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
        # split into input (X) and output (Y) variables
        X = dataset[:,0:8]
        Y = dataset[:,8]
        # create model
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Fit the model
        model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
        # evaluate the model
        scores = model.evaluate(X, Y, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        return X, Y, model
    except Exception as e:
        print(e)
        return None

    
def write_file(path, string_values):
    """
    Write a string 
    """
    with open(path, "w") as file:
        file.write(string_values)
    print('Model saved to disk')


def read_file(path):
    with open(path, 'r') as file:
        return file.read()
    print("Loaded model from disk")


# https://stackoverflow.com/a/43374773/2108769
def scan_hdf5(path, recursive=True, tab_step=2):
    def scan_node(g, tabs=0):
        print(' ' * tabs, g.name)
        for k, v in g.items():
            if isinstance(v, h5py.Dataset):
                print(' ' * tabs + ' ' * tab_step + ' -', v.name)
            elif isinstance(v, h5py.Group) and recursive:
                scan_node(v, tabs=tabs + tab_step)
    with h5py.File(path, 'r') as f:
        scan_node(f)

## Create Model

In [4]:
X, Y, model = create_model()


model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 77.60%


## Saving full model in HDF5 (Weights + NN Architecture)

> You can use model.save(filepath) to save a Keras model into a single [HDF5](https://www.hdfgroup.org/) file which will contain:

> - the architecture of the model, allowing to re-create the model
> - the weights of the model
> - the training configuration (loss, optimizer)
> - the state of the optimizer, allowing to resume training exactly where you left off.

> You can then use keras.models.load_model(filepath) to reinstantiate your model. load_model will also take care of compiling the model using the saved training configuration (unless the model was never compiled in the first place).

In [5]:
full_model = model
full_model.save('full-model.h5')
print("Saved model to disk")

Saved model to disk


In [6]:
del full_model

In [7]:
full_model

NameError: name 'full_model' is not defined

In [9]:
from keras.models import load_model


full_model = load_model('full-model.h5')

score = full_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (full_model.metrics_names[1], score[1]*100))

accuracy: 77.60%


In [10]:
# Scanning the file

scan_hdf5('full-model.h5') # Arch + Weights

 /
   /model_weights
     /model_weights/dense_1
       /model_weights/dense_1/dense_1
         - /model_weights/dense_1/dense_1/bias:0
         - /model_weights/dense_1/dense_1/kernel:0
     /model_weights/dense_2
       /model_weights/dense_2/dense_2
         - /model_weights/dense_2/dense_2/bias:0
         - /model_weights/dense_2/dense_2/kernel:0
     /model_weights/dense_3
       /model_weights/dense_3/dense_3
         - /model_weights/dense_3/dense_3/bias:0
         - /model_weights/dense_3/dense_3/kernel:0
   /optimizer_weights
     /optimizer_weights/Adam
       - /optimizer_weights/Adam/iterations:0
     /optimizer_weights/training
       /optimizer_weights/training/Adam
         - /optimizer_weights/training/Adam/m_0_1:0
         - /optimizer_weights/training/Adam/m_1_1:0
         - /optimizer_weights/training/Adam/m_2_1:0
         - /optimizer_weights/training/Adam/m_3_1:0
         - /optimizer_weights/training/Adam/m_4_1:0
         - /optimizer_weights/training/Adam/m_5_1:0

In [11]:
!head -n 155 full-model.h5



                    ���������w      ��������        `              �       �                                     TREE   ����������������                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      HEAP    X       0       �              model_weights   optimizer_weights              (                                     �       �       H        keras_version                                          @        backend                          
   model_config                     

### Saving model's architecture to JSON and model's architecture in H5

#### Arch

In [12]:
arch_json = model.to_json()

In [13]:
type(arch_json)

str

In [14]:
arch_json

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "acti

In [15]:
write_file('arch.json', arch_json)

Model saved to disk


In [16]:
!cat arch.json

{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activ

#### Weights

In [17]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")
print("Saved weights model to disk")

Saved weights model to disk


In [18]:
!head -n 155 model_weights.h5



                    ���������=      ��������        `              �       �                                      TREE   ����������������                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      HEAP    X               �              dense_1 dense_2 dense_3        8                                                     �       �       P        layer_names                             dense_1dense_2dense_3    @        backend                          
   weight_names             

In [19]:
scan_hdf5('model_weights.h5')

 /
   /dense_1
     /dense_1/dense_1
       - /dense_1/dense_1/bias:0
       - /dense_1/dense_1/kernel:0
   /dense_2
     /dense_2/dense_2
       - /dense_2/dense_2/bias:0
       - /dense_2/dense_2/kernel:0
   /dense_3
     /dense_3/dense_3
       - /dense_3/dense_3/bias:0
       - /dense_3/dense_3/kernel:0


#### A few moments later...

In [20]:
from keras.models import model_from_json



model_fromjsonh5 = model_from_json(read_file('arch.json'))



In [21]:
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [22]:
model_fromjsonh5.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 64.58%


In [23]:
model_fromjsonh5.load_weights("model_weights.h5")

In [24]:
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 77.60%


In [25]:
model_fromjsonh5.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model_fromjsonh5.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_fromjsonh5.metrics_names[1], score[1]*100))

accuracy: 77.60%


------

### Serializing model's architecture to YAML and model's architecture in HDF5


#### Arch

In [26]:
arch_yaml = model.to_yaml()


write_file('arch.yaml', arch_yaml)

Model saved to disk


In [27]:
print(arch_yaml)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 8
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config:
          distribution: uniform
          mode: fan_avg
          scale: 1.0
          seed: null
      kernel_regularizer: null
      name: dense_1
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceSc

In [28]:
!cat arch.yaml

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 8
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config:
          distribution: uniform
          mode: fan_avg
          scale: 1.0
          seed: null
      kernel_regularizer: null
      name: dense_1
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceSc

In [29]:
# serialize weights to HDF5
model.save_weights("model_weights.h5")
print("Saved weights model to disk")

Saved weights model to disk


In [30]:
scan_hdf5('model_weights.h5')

 /
   /dense_1
     /dense_1/dense_1
       - /dense_1/dense_1/bias:0
       - /dense_1/dense_1/kernel:0
   /dense_2
     /dense_2/dense_2
       - /dense_2/dense_2/bias:0
       - /dense_2/dense_2/kernel:0
   /dense_3
     /dense_3/dense_3
       - /dense_3/dense_3/bias:0
       - /dense_3/dense_3/kernel:0


### Load YAML and H5 model

In [31]:
# model reconstruction from YAML
from keras.models import model_from_yaml
model_yaml = model_from_yaml(read_file('arch.yaml'))


In [32]:
# load weights
model_yaml.load_weights("model_weights.h5")

In [33]:
# Compile before predictions
score = model_yaml.evaluate(X, Y, verbose=0)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [34]:
# evaluate loaded model on test data
model_yaml.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model_yaml.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_yaml.metrics_names[1], score[1]*100))

accuracy: 77.60%


-----


# Other Nice hacks....



Pickle is not recommended by the keras team (https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model), but nobody said a word about Joblib.

In [35]:
joblib.dump(model, 'model.joblib')

['model.joblib']

In [36]:
model_job_lib = joblib.load('model.joblib')

In [38]:
score = model_job_lib.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model_job_lib.metrics_names[1], score[1]*100))

accuracy: 77.60%


In [39]:
dir(model_job_lib)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_base_init',
 '_build_input_shape',
 '_built',
 '_cache_output_metric_attributes',
 '_check_trainable_weights_consistency',
 '_collected_trainable_weights',
 '_compile_metric_functions',
 '_compile_metrics',
 '_compile_weighted_metrics',
 '_compute_previous_mask',
 '_expects_training_arg',
 '_feed_input_names',
 '_feed_input_shapes',
 '_feed_inputs',
 '_feed_loss_fns',
 '_feed_output_names',
 '_feed_output_shapes',
 '_feed_outputs',
 '_feed_sample_weight_modes',
 '_feed_sample_weights',
 '_feed_targets',
 '_function_kwargs',


---

# Other tools for Model Persistence


- [Open Neural Network Exchange (ONNX)](https://onnx.ai/)
- [The Predictive Model Markup Language (PMML)](http://dmg.org/pmml/v4-4/GeneralStructure.html)



# Platforms for Model Serving

- [Polyaxon](https://polyaxon.com/)
- [TensorFlow Extended](https://www.tensorflow.org/tfx)
- [KubeFlow](https://www.kubeflow.org/)
- [MLFlow](https://mlflow.org/)